In [ ]:
# default_exp utils

# Utils

> General utilities. Should probably split up into `utils.time` and `utils.download`

In [ ]:
#hide
from nbverbose.showdoc import show_doc  # noqa

In [ ]:
# export
import datetime as dt
import email.utils as eut
import http.client as httplib
import logging
from typing import Union, Tuple
from math import radians, tan
from pathlib import Path
from urllib.request import urlopen

import requests
from tqdm.auto import tqdm

import pandas as pd

logger = logging.getLogger(__name__)
try:
    from osgeo import gdal
except ImportError:
    GDAL_INSTALLED = False
    logger.warning(
        "No GDAL found. Some planetary.utils functions not working, but okay."
    )
else:
    GDAL_INSTALLED = True

## Time format strings

These are the different format strings these utils convert from and to.

An identifier with `xxx_dt_format_xxx` in its name signifies a full `datetime` format as compared to dates only.

In [ ]:
# export
nasa_date_format = "%Y-%j"
nasa_dt_format = nasa_date_format + "T%H:%M:%S"
nasa_dt_format_with_ms = nasa_dt_format + ".%f"
standard_date_format = "%Y-%m-%d"
standard_dt_format = standard_date_format + "T%H:%M:%S"
standard_dt_format_with_ms = standard_dt_format + ".%f"

In [ ]:
# export
def nasa_date_to_iso(
    datestr: str,  # Date string of the form Y-j
    with_hours: bool = False,  # Switch if return is wanted with hours (i.e. isoformat)
) -> str:  # Datestring in either Y-m-d or ISO-format.
    "Convert the day-number based NASA date format to ISO."
    date = dt.datetime.strptime(datestr, nasa_date_format)
    if with_hours:
        return date.isoformat()
    else:
        return date.strftime(standard_date_format)

In [ ]:
nasa_date = "2010-110"
iso_date = "2010-4-20"

In [ ]:
assert nasa_date_to_iso(nasa_date, with_hours=True) == "2010-04-20T00:00:00"
assert nasa_date_to_iso(nasa_date) == "2010-04-20"

In [ ]:
# export
def iso_to_nasa_date(
    datestr: str,  # Date string of the form Y-m-d
) -> str:  # Datestring in NASA standard yyyy-jjj
    "Convert iso date to day-number based NASA date."
    date = dt.datetime.strptime(datestr, standard_date_format)
    return date.strftime(nasa_date_format)

In [ ]:
assert iso_to_nasa_date(iso_date) == nasa_date

In [ ]:
# export
def nasa_datetime_to_iso(
    dtimestr: str,  # Datetime string of the form Y-jTH-M-S
):  # Datestring in ISO standard yyyy-mm-ddTHH:MM:SS.xxxxxx
    """Convert the day-number based NASA datetime format to ISO.

    Note: This is dateTIME vs `nasa_date_to_iso` which is just DATE.
    """
    try:
        dtimestr.split(".")[1]
    except IndexError:
        source_format = nasa_dt_format
    else:
        source_format = nasa_dt_format_with_ms
    time = dt.datetime.strptime(dtimestr, source_format)
    return time.isoformat()

In [ ]:
nasa_datetime = "2010-110T10:12:14"
nasa_datetime_with_ms = nasa_datetime + ".123000"
iso_datetime = "2010-04-20T10:12:14"
iso_datetime_with_ms = iso_datetime + ".123000"

In [ ]:
assert nasa_datetime_to_iso(nasa_datetime) == iso_datetime
assert nasa_datetime_to_iso(nasa_datetime_with_ms) == iso_datetime_with_ms

In [ ]:
# export
def iso_to_nasa_datetime(
    dtimestr: str,  # Datetime string of the form yyyy-mm-ddTHH-MM-SS
):  # Datestring in NASA standard yyyy-jjjTHH-MM-SS
    "Convert iso datetime to day-number based NASA datetime."
    try:
        dtimestr.split(".")[1]
    except IndexError:
        source_format = standard_dt_format
        target_format = nasa_dt_format
    else:
        source_format = standard_dt_format_with_ms
        target_format = nasa_dt_format_with_ms
    date = dt.datetime.strptime(dtimestr, source_format)
    return date.strftime(target_format)

In [ ]:
assert iso_to_nasa_datetime(iso_datetime) == nasa_datetime
assert iso_to_nasa_datetime(iso_datetime_with_ms) == nasa_datetime_with_ms

In [ ]:
# export
def replace_all_nasa_times(
    df: pd.DataFrame,  # DataFrame with NASA time columns
):
    """Find all NASA times in dataframe and replace with ISO.

    Changes will be implemented on incoming dataframe!

    This will be done for all columns with the word TIME in the column name.
    """
    for col in [col for col in df.columns if "TIME" in col]:
        if "T" in df[col].iloc[0]:
            df[col] = pd.to_datetime(df[col].map(nasa_datetime_to_iso))

## Network utils

In [ ]:
# export
def parse_http_date(
    text: str,  # datestring from urllib.request
) -> dt.datetime:  # dt.datetime object from given datetime string
    "Parse date string retrieved via urllib.request."
    return dt.datetime(*eut.parsedate(text)[:6])


def get_remote_timestamp(
    url: str,  # URL to check timestamp for
) -> dt.datetime:
    """Get the timestamp of a remote file.

    Useful for checking if there's an updated file available.
    """
    with urlopen(url, timeout=10) as conn:
        t = parse_http_date(conn.headers["last-modified"])
    return t


def url_retrieve(
    url: str,  # The URL to download
    outfile: str,  # The path where to store the downloaded file.
    # The size of the chunk for the request.iter_content call. Default: 128
    chunk_size: int = 128,
):
    """Improved urlretrieve with progressbar, timeout and chunker.

    This downloader has built-in progress bar using tqdm and using the `requests`
    package it improves standard `urllib` behavior by adding time-out capability.

    I tested different chunk_sizes and most of the time 128 was actually fastest, YMMV.

    Inspired by https://stackoverflow.com/a/61575758/680232
    """
    R = requests.get(url, stream=True, allow_redirects=True)
    if R.status_code != 200:
        raise ConnectionError(f"Could not download {url}\nError code: {R.status_code}")
    with tqdm.wrapattr(
        open(outfile, "wb"),
        "write",
        miniters=1,
        total=int(R.headers.get("content-length", 0)),
        desc=str(Path(outfile).name) + "\n",
    ) as fd:
        for chunk in R.iter_content(chunk_size=chunk_size):
            fd.write(chunk)


def have_internet():
    """Fastest way to check for active internet connection.

    From https://stackoverflow.com/a/29854274/680232
    """
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        return True
    except:
        conn.close()
        return False

## Image processing helpers

In [ ]:
# export
def height_from_shadow(
    shadow_in_pixels: float,  # Measured length of shadow in pixels
    sun_elev: float,  # Ange of sun over horizon in degrees
) -> float:  # Height [meter]
    """Calculate height of an object from its shadow length.

    Note, that your image might have been binned.
    You need to correct `shadow_in_pixels` for that.
    """
    return tan(radians(sun_elev)) * shadow_in_pixels


def get_gdal_center_coords(
    imgpath: Union[str, Path],  # Path to raster image that is readable by GDLA
) -> Tuple[int, int]:  # center row/col coordinates.
    """Get center rows/cols pixel coordinate for GDAL-readable dataset.

    Check CLI `gdalinfo --formats` to see all formats that GDAL can open.
    """
    if not GDAL_INSTALLED:
        logger.error("GDAL not installed. Returning")
        return
    ds = gdal.Open(str(imgpath))
    xmean = ds.RasterXSize // 2
    ymean = ds.RasterYSize // 2
    return xmean, ymean


def file_variations(
    filename: Union[str, Path],  # The original filename to use as a base.
    extensions: list
) -> list:  # list of Paths
    """Create a variation of file names.

    Generate a list of variations on a filename by replacing the extension with
    the provided list.

    Adapted from T. Olsens `file_variations of the pysis module for using pathlib.
    """
    return [Path(filename).with_suffix(extension) for extension in extensions]

In [ ]:
fname = 'abc.txt'

In [ ]:
extensions = ".cub .cal.cub .map.cal.cub".split()

In [ ]:
file_variations(fname, extensions)

[Path('abc.cub'), Path('abc.cal.cub'), Path('abc.map.cal.cub')]

In [ ]:
assert len(extensions) == len(file_variations(fname, extensions))